# **Otimização de Portifólio**

In [1]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import cvxpy as cp

In [2]:
acoes = [
    'AAPL',  # Apple
    'MSFT',  # Microsoft
    'GOOGL', # Alphabet (Google)
    'AMZN',  # Amazon
    'META',  # Meta (Facebook)
    'JPM',   # JPMorgan Chase
    'BAC',   # Bank of America
    'XOM',   # Exxon Mobil
    'CVX',   # Chevron
    'V',     # Visa
    'MA',    # Mastercard
    'PG',    # Procter & Gamble
    'KO',    # Coca-Cola
    'PEP',   # PepsiCo
    'DIS',   # Disney
    'ADBE',  # Adobe
    'ORCL'   # Oracle
]

# Definindo o período e o intervalo
periodo = "1y"  # Último ano
intervalo = "1d"  # Dados diários

# Dicionário para armazenar os dados de cada ação
dados_acoes = {}

# Loop para baixar os dados de cada ação
for acao in acoes:
    try:
        # Download dos dados históricos
        dados = yf.download(acao, period=periodo, interval=intervalo)
        dados_acoes[acao] = dados["Close"]
    except Exception as e:
        print(f"Erro ao baixar dados para {acao}: {e}")

# Convertendo os dados em um único DataFrame para análise
historico_completo = pd.concat(dados_acoes, axis=1)

# Encontrando o retorno diário em porcentagem
historico_completo = historico_completo.diff()/historico_completo.shift(1)

# Multiplicando o retorno por 100
historico_completo = 100 * historico_completo

# Retirando NAs
historico_completo = historico_completo.dropna()

# Encontrando média e variância
media = np.mean(historico_completo, axis=0).values
cov = np.cov(historico_completo.T)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## Problema deminimização de variância

\begin{aligned}
\text{Minimize:} \quad & \mathbf{w}^\top \Sigma \mathbf{w} \\\\
\text{Subject to:} \quad & \mathbf{w}^\top \mathbf{1} = 1 \\\\
& \mathbf{w} \geq 0 \\\\:
\end{aligned}

In [3]:
# Número de ativos
n = len(media)

# Variável de decisão: pesos do portfólio
w = cp.Variable(n)

# Função objetivo: minimizar o risco (variância do portfólio)
risco = cp.quad_form(w, cov)  # w^T * cov * w
objetivo = cp.Minimize(risco)

# Restrição: os pesos devem somar 1
soma_pesos = cp.sum(w) == 1

# Restrição: os pesos devem ser não-negativos (opcional para portfólios long-only)
pesos_nao_negativos = w >= 0

# Configuração do problema de otimização
problema = cp.Problem(objetivo, [soma_pesos, pesos_nao_negativos])

# Resolver o problema
problema.solve()

# Resultados
print("Status da solução:", problema.status)
print("Pesos ótimos do portfólio:", w.value)
print("Risco mínimo:", risco.value)


Status da solução: optimal
Pesos ótimos do portfólio: [6.48592866e-02 8.12250148e-03 4.36428847e-02 1.97831272e-02
 2.01980476e-02 2.83583183e-02 1.83006986e-23 1.09867802e-01
 7.52368149e-02 1.91966820e-02 1.73987358e-02 2.46101787e-01
 2.29396679e-01 3.07182279e-02 4.46693060e-02 8.54966372e-03
 3.39001360e-02]
Risco mínimo: 0.28335860028310045


## Problema de minimização de variância sujeito à um retorno mínimo

\begin{aligned}
\text{Minimize:} \quad & \mathbf{w}^\top \Sigma \mathbf{w} \\\\
\text{Subject to:} \quad & \mathbf{w}^\top \mathbf{1} = 1 \\\\
& \mathbf{w} \geq 0 \\\\
& \mathbf{w}^\top \mathbf{\mu} \geq R_{\text{min}}
\end{aligned}

In [4]:
# Número de ativos
n = len(media)

# Variável de decisão: pesos do portfólio
w = cp.Variable(n)

# Função objetivo: minimizar o risco (variância do portfólio)
risco = cp.quad_form(w, cov)  # w^T * cov * w
objetivo = cp.Minimize(risco)

# Restrição: os pesos devem somar 1
soma_pesos = cp.sum(w) == 1

# Restrição: os pesos devem ser não-negativos (opcional para portfólios long-only)
pesos_nao_negativos = w >= 0

# Restrição: o retorno esperado do portfólio deve ser pelo menos 1.2^(1/250)
retorno_minimo = cp.sum(cp.multiply(media, w)) >= 100*(1.3**(1/250)-1)

# Configuração do problema de otimização
problema = cp.Problem(objetivo, [soma_pesos, pesos_nao_negativos, retorno_minimo])

# Resolver o problema
problema.solve()

# Resultados
print("Status da solução:", problema.status)
print("Pesos ótimos do portfólio:", w.value)
print("Risco mínimo:", risco.value)

Status da solução: optimal
Pesos ótimos do portfólio: [ 4.49777466e-02 -7.62329432e-23  4.17482557e-02  1.55298923e-02
  8.00125103e-02  1.84798736e-01  1.27159854e-24  9.73853375e-02
 -4.07313421e-23 -4.04703359e-23  6.76724461e-04  2.11191387e-01
  2.00758940e-01 -9.16866162e-23  3.44655864e-02 -1.02826608e-22
  8.84548842e-02]
Risco mínimo: 0.3732297542071406


## Problema de Maximização de retorno sujeito à uma variância máxima

\begin{aligned}
\text{Maximize:} \quad & \mathbf{w}^\top \mathbf{\mu} \\\\
\text{Subject to:} \quad & \mathbf{w}^\top \mathbf{1} = 1 \\\\
& \mathbf{w} \geq 0 \\\\
& \mathbf{w}^\top \Sigma \mathbf{w} \leq \sigma_{\text{max}}^2
\end{aligned}

In [5]:
# Número de ativos
n = len(media)

# Variável de decisão: pesos do portfólio
w = cp.Variable(n)

# Função objetivo: maximizar o retorno esperado do portfólio
retorno_esperado = cp.sum(cp.multiply(media, w))
objetivo = cp.Maximize(retorno_esperado)

# Restrição: os pesos devem somar 1
soma_pesos = cp.sum(w) == 1

# Restrição: os pesos devem ser não-negativos (opcional para portfólios long-only)
pesos_nao_negativos = w >= 0

# Restrição: a variância do portfólio deve ser menor ou igual a um limite máximo
variancia_maxima = 0.30  # Substitua pelo valor desejado
restricao_variancia = cp.quad_form(w, cov) <= variancia_maxima

# Configuração do problema de otimização
problema = cp.Problem(objetivo, [soma_pesos, pesos_nao_negativos, restricao_variancia])

# Resolver o problema
problema.solve()

# Resultados
print("Status da solução:", problema.status)
print("Pesos ótimos do portfólio:", w.value)
print("Retorno máximo:", retorno_esperado.value)


Status da solução: optimal
Pesos ótimos do portfólio: [5.79802596e-02 3.09816593e-09 4.44538129e-02 2.10670733e-02
 4.51205310e-02 9.34291241e-02 5.01229167e-08 1.11407285e-01
 3.73285994e-02 7.94333700e-03 1.49711434e-02 2.36071977e-01
 2.30295287e-01 7.88683018e-10 4.12858903e-02 6.74185100e-10
 5.86456258e-02]
Retorno máximo: 0.08205385187696483
